### 1. 패키지 설치

In [ ]:
pip install --upgrade pip

In [ ]:
pip install -r requirements.txt  # 패키지 설치

In [1]:
import torch
print(torch.__version__)  # PyTorch 버전
print(torch.version.cuda)  # PyTorch가 사용하는 CUDA 버전
print(torch.cuda.is_available())  # GPU 사용 가능 여부


2.4.1+cu121
12.1
True


### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [2]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

/home/chae/env_0217/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로그인 상태입니다. 사용자: chaeeee


In [3]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [1]:
from langchain.schema import Document
import os, json
from tqdm import tqdm

# 1. JSON 파일 경로 설정
json_path = "./json_data.json"  # 단일 JSON 파일 경로

# 2. JSON 데이터 불러오기
with open(json_path, 'r', encoding='utf-8') as f:
    all_data = json.load(f)

# 3. {}를 기준으로 JSON 데이터 분할 및 Document 객체 생성
documents = []

for data in tqdm(all_data, desc="Generating Documents", unit="entry", ncols=80):
    metadata = {
        "title": data.get('title', 'N/A'),
        "artist": data.get('artist', 'N/A'),
        "year": data.get('year', 'N/A'),
        "read_count": data.get('read_count', 0),
        "collection": data.get('collection')
    }

    # JSON 데이터의 각 항목을 Document 객체로 변환
    doc_content = json.dumps(data, ensure_ascii=False, indent=4)
    documents.append(Document(
        page_content=doc_content.strip(),
        metadata=metadata
    ))

Generating Documents: 100%|█████████| 11479/11479 [00:00<00:00, 71299.71entry/s]


In [2]:
len(documents)

11479

In [3]:
documents[1]

Document(metadata={'title': '팔괘호', 'artist': '한기석', 'year': '1960', 'read_count': 175, 'collection': '국립현대미술관'}, page_content='{\n    "title": "팔괘호",\n    "title_eng": "Palgwae Vase",\n    "artist": "한기석",\n    "artist_eng": "HAN Kisuk",\n    "artwork_number": 2,\n    "year": "1960",\n    "size": "250×127",\n    "materials": "캔버스, 종이에 유화 물감",\n    "category": "회화 II",\n    "description": "한농(韓農) 한기석(1930-2011)은 표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는 항아리와 화병을 계속 그렸는데, 항아리에 대한 사람들의 일반적 향수 이미지는 그의 작품에서 친근감을 불어넣어 준다. 또한 완벽한 균형을 이루면서 노련하게 표현되어 다양한 색의 조화를 지닌 도자기의 미를 느낄 수 있다.",\n    "read_count": 175,\n    "collection": "국립현대미술관"\n}')

In [7]:
# 특정 Document 객체의 텍스트 길이 확인
len(documents[11001].page_content)

1152

In [8]:
# 800자를 초과하는 Document 개수 세기
over_800_count = sum(1 for doc in documents if len(doc.page_content) > 800)

# 결과 출력
print(f"800자를 초과하는 Document 개수: {over_800_count}")


800자를 초과하는 Document 개수: 6971


In [2]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 1. 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


/home/chae/env_0217/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
You try to use a model that was created with version 3.3.1, however, your version is 3.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [5]:
# 2. 문서 데이터와 메타데이터 분리
texts = [doc.page_content for doc in documents]  # 문서 텍스트
metadatas = [doc.metadata for doc in documents]  # 문서 메타데이터

# 3. 문서 임베딩 생성
embeddings = embedding_model.encode(texts)

In [6]:
import numpy as np

# 4. FAISS 인덱스 생성
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(embeddings.astype(np.float32))   


In [8]:
from langchain.docstore.in_memory import InMemoryDocstore

docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})

# 6. index_to_docstore_id도 str로!
index_to_docstore_id = {i: str(i) for i in range(len(documents))}

# 7. FAISS 벡터스토어 생성
def embed_query(text):
    return embedding_model.encode([text])[0]

faiss_db = FAISS(
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id,
    embedding_function=embed_query
)

# 7. FAISS 데이터베이스 저장
faiss_db.save_local("./faiss_artworks_0324_json")
print("FAISS 데이터베이스가 성공적으로 저장되었습니다!")


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 저장되었습니다!


In [14]:
# 9. 검색 테스트
query = "박승무의 설경"
results = faiss_db.similarity_search(query, k=5)

# 10. 검색 결과 출력
for result in results:
    print("문서 텍스트:", result.page_content)
    print("문서 메타데이터:", result.metadata)
    print('=------------------------------------------')


문서 텍스트: {
    "title": "설경(雪景)",
    "title_eng": "Snowy Landscape",
    "artist": "박승무",
    "artist_eng": "PARK Seungmoo",
    "artwork_number": 9402,
    "year": "1965",
    "size": "157×157",
    "materials": "종이에 먹, 색",
    "category": "회화 I",
    "description": "심향(深香) 박승무(朴勝武, 1893-1980)는 1913년부터 서화미술회의 안중식(安中植, 1861-1919)과 조석진(趙錫晋, 1853-1920) 문하에서 전통화법을 배웠으며, 졸업한 이듬해 중국 상하이로 향했으나 1919년 3·1 운동 이후 귀국했다. 《조선미술전람회》와 《서화협회전람회》에 출품하며 활동을 시작하였고, 광복 이후에는 대전에 정착하여 작업을 지속했다. 1957년 제1회 충청남도 문화상 미술부문 수상자로 선정되었다.<설경>은 눈 온 후의 산촌 풍경을 담은 산수화로, 박승무가 즐겨 그렸던 주제 중 하나이다. 작가는 화면 왼쪽에 커다란 산을 배치하고, 오른쪽으로는 공간을 시원하게 열어주는 구도를 취했다. 근경의 지팡이를 든 노인과 아이는 박승무의 산수화에 자주 등장하는 인물상으로, 점경인물과 가옥이 어우러져 정감있게 표현되었다.",
    "read_count": 59,
    "collection": "국립현대미술관"
}
문서 메타데이터: {'title': '설경(雪景)', 'artist': '박승무', 'year': '1965', 'read_count': 59, 'collection': '국립현대미술관'}
=------------------------------------------
문서 텍스트: {
    "title": "설경",
    "title_eng": "Snowscape",
    "artist": "박승무",
    "artist_eng": "PARK Se

In [10]:
import pickle

index_pkl_path = "./faiss_artworks_0324_json/index.pkl"

# index.pkl 파일을 직접 열어서 확인
with open(index_pkl_path, "rb") as f:
    data = pickle.load(f)

print(f"📌 index.pkl 내용: {type(data)}, 길이: {len(data)}")

if isinstance(data, tuple):
    print(f"📌 데이터 내부 타입: {[type(item) for item in data]}")


📌 index.pkl 내용: <class 'tuple'>, 길이: 2
📌 데이터 내부 타입: [<class 'langchain_community.docstore.in_memory.InMemoryDocstore'>, <class 'dict'>]


In [31]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_merged_final"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    print('faiss_db 로드 성공')
except Exception as e:
    print(f"📌 FAISS.load_local() 반환값: {type(faiss_db)}")
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


faiss_db 로드 성공


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bit 양자화 활성화
    bnb_4bit_compute_dtype=torch.float16,  # 계산 타입 설정 (float16이 일반적)
    bnb_4bit_use_double_quant=True,  # 더블 양자화 사용 (메모리 절약)
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) 사용
)

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit 양자화 활성화
    bnb_4bit_compute_dtype="float16",  # 계산 정밀도 설정
    bnb_4bit_quant_type="nf4",  # NF4 양자화 방식 사용 (효율적)
    bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
)

In [6]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # ✅ 올바른 양자화 설정 적용
    device_map="auto",  # ✅ 자동 GPU 배치
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [7]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

NameError: name 'model' is not defined

In [8]:
from langchain.prompts import ChatPromptTemplate

deepseek_template = """
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions **accurately based on the provided document (context)**. 
If the requested information is not found in the document, respond with:
"문서에 해당 정보가 없습니다." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, **bold** or *italic* text) to enhance readability.
You must include the artwork number in your response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
"""




exaone_template = '''
<|system|>
You are an AI assistant tasked with refining and polishing the provided logical reasoning into a final answer in Korean.  
Your role is to produce a clear, concise, and well-structured response that maintains the original meaning and key details.  
Ensure that your final answer is written in Korean and uses **beautiful Markdown formatting** (e.g., headings, bullet points, **bold** or *italic* text) to enhance readability.  
Focus solely on refining the content without adding any new information.
You must include the artwork number in your response.

<|reasoning|>
{reasoning}

<|user|>
Based on the above reasoning, please generate a refined and final answer in Korean.

<|assistant|>
'''



# DeepSeek 템플릿 생성
deepseek_prompt = ChatPromptTemplate.from_template(deepseek_template)

# EXAONE 템플릿 생성
exaone_prompt = ChatPromptTemplate.from_template(exaone_template)



In [9]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, bold or italic text) to enhance readability.
You must include artwork number.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [10]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.3      # 다양성과 관련성 간 균형
    }
)


In [11]:
import re

class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"</think>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [12]:
import re

class MarkdownOutputParser2:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [13]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 🔹 EXAONE 모델 로드
exaone_model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
exaone_tokenizer = AutoTokenizer.from_pretrained(exaone_model_id)
exaone_model = AutoModelForCausalLM.from_pretrained(
    exaone_model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:24<00:00,  3.55s/it]


In [14]:
from transformers import pipeline

# 파이프라인 생성
exaone_pipe = pipeline(
    "text-generation",
    model=exaone_model,
    tokenizer=exaone_tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.2,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)

/tmp/ipykernel_43857/1623051539.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)


In [18]:
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable import RunnablePassthrough, RunnableMap

chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | exaone_llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser2()                    # 응답을 문자열로 변환
)

In [ ]:
from langchain.schema.runnable import RunnableLambda

chain = (
    retriever
    | RunnableLambda(lambda docs: {  
        "context": "\n".join([doc.page_content for doc in docs]),  
        "question": query 
    })
    | deepseek_prompt
    | llm
    | MarkdownOutputParser()
    | (lambda x: {"reasoning": x})
    | exaone_prompt
    | exaone_llm
    | MarkdownOutputParser2()
)


NameError: name 'llm' is not defined

In [69]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0324_pdf2"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    print('faiss_db 로드 성공')
except Exception as e:
    print(f"📌 FAISS.load_local() 반환값: {type(faiss_db)}")
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


faiss_db 로드 성공


In [70]:
print("🧩 index 벡터 수:", faiss_db.index.ntotal)
print("🧩 docstore 문서 수:", len(faiss_db.docstore._dict))
print("🧩 index_to_docstore_id:", len(faiss_db.index_to_docstore_id))


🧩 index 벡터 수: 746
🧩 docstore 문서 수: 746
🧩 index_to_docstore_id: 746


In [71]:
print(type(list(faiss_db.index_to_docstore_id.values())[0]))  # 예: <class 'str'> 나 <class 'int'>?

<class 'str'>


In [65]:
query = "미술품 수집의 성과는?" # 746, 12403

In [66]:
response = chain.invoke({"question": query})
print(response)

KeyError: 12593

In [32]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


Document 1:
Content: {
    "title": "소리 없는 소리",
    "title_eng": "The Silent Sound ",
    "artist": "김은설",
    "artist_eng": "KIM Eunseol",
    "artwork_number": 11301,
    "year": "N/A",
    "size": "5분 5초",
    "materials": "3채널 영상, 컬러, 사운드",
    "category": "뉴미디어",
    "description": "김은설(金珢雪, 1988- )은 청주대학교에서 회화를 전공한 후 회화, 드로잉, 영상 설치, 퍼포먼스 등 다양한 매체를 다루며 작업하고 있다. 작가는 주로 소리를 촉각이나 시각으로 표현하는 등 청각을 또 다른 감각으로 변환하여 다룸으로써 신체 감각과 소통에 대한 본질적인 질문을 던지는 작업에 집중하고 있다. 2024년 《여기 닿은 노래》(아르코미술관), 2022년 《듣다 보다》(JCC아트센터) 등 다수 전시에 참여한 바 있으며, 2023년 <체록(體錄)-다른 존재의 몸짓을 내 몸으로 기록하기>(청년예술청 SAPY 그레이룸) 공연, 2021년부터 진행한 <므브프> 프로젝트 등 다양한 유형의 활동을 지속하고 있다.<소리 없는 소리>는 작가가 일상에서 흥미로운 소리가 날 것 같은 장면들을 촬영한 후, 해당 장면에서 느껴지는 촉감이나 예상되는 소리를 수집하여 영상에 덧입힌 작품이다. 작가는 항상 보청기를 통해 교정된 소리를 듣기 때문에 본인이 감각하는 청각이 불완전하다는 의식으로 일상에서는 청각에만 의존하지 않고 눈으로 본 형상을 통해 소리를 본다고 한다. 작품은 작가 스스로 ‘감각하는 소리’와 ‘보는 소리’ 사이의 일치화 과정을 담고 있으며, 영상과 사운드의 일치 혹은 불일치를 통해 관객에게 듣는다는 것이 무엇인지, 나아가 감각한다는 것의 본질에 대해 재고하게 한다.",
    "read_count": 66,
    "collection": "국립현대미술관"

/tmp/ipykernel_1049/3695306690.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [ ]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


### PDF DB에 추가

In [20]:
import os
import pickle
import faiss
import numpy as np

def load_faiss_index(db_path):
    """FAISS 인덱스 로드"""
    index = faiss.read_index(os.path.join(db_path, "index.faiss"))
    with open(os.path.join(db_path, "index.pkl"), "rb") as f:
        docstore = pickle.load(f)
    
    # docstore가 튜플이면 첫 번째 요소만 사용
    if isinstance(docstore, tuple):
        docstore = docstore[0]
    
    # mapping을 index.pkl에서 로드 (index_to_docstore_id.pkl이 없음)
    mapping = {i: str(i) for i in range(index.ntotal)}
    
    return index, docstore, mapping

def merge_faiss_databases(db1_dir, db2_dir, merged_dir):
    """두 개의 FAISS 데이터베이스 병합"""
    index1, docstore1, mapping1 = load_faiss_index(db1_dir)
    index2, docstore2, mapping2 = load_faiss_index(db2_dir)

    dim = index1.d  # 첫 번째 인덱스의 차원을 자동 감지
    merged_index = faiss.IndexFlatL2(dim)
    merged_docstore = {}
    merged_mapping = {}

    # 첫 번째 DB 추가
    num_vectors1 = index1.ntotal
    for i in range(num_vectors1):
        vec = np.array(index1.reconstruct(i)).reshape(1, dim).astype("float32")
        merged_index.add(vec)
        merged_mapping[i] = mapping1[i]

    # 두 번째 DB 추가 (ID 오프셋 적용)
    num_vectors2 = index2.ntotal
    offset = num_vectors1
    for i in range(num_vectors2):
        vec = np.array(index2.reconstruct(i)).reshape(1, dim).astype("float32")
        merged_index.add(vec)
        merged_mapping[i + offset] = mapping2[i]

    # 문서 저장소 병합
    assert isinstance(docstore1, dict), f"docstore1가 dict가 아닙니다: {type(docstore1)}"
    merged_docstore.update(docstore1)
    assert isinstance(docstore2, dict), f"docstore2가 dict가 아닙니다: {type(docstore2)}"
    merged_docstore.update(docstore2)

    # 결과 저장
    os.makedirs(merged_dir, exist_ok=True)
    faiss.write_index(merged_index, os.path.join(merged_dir, "index.faiss"))
    with open(os.path.join(merged_dir, "index.pkl"), "wb") as f:
        pickle.dump(merged_docstore, f)
    with open(os.path.join(merged_dir, "index_to_docstore_id.pkl"), "wb") as f:
        pickle.dump(merged_mapping, f)
    print(f"병합된 FAISS 데이터베이스 저장 완료: {merged_dir}")

# 예제 실행
if __name__ == "__main__":
    db1_dir = "./faiss_artworks_0304"  # 첫 번째 FAISS DB 디렉토리
    db2_dir = "./faiss_artworks_0318"  # 두 번째 FAISS DB 디렉토리
    merged_dir = "./faiss_merged_db"    # 병합된 결과를 저장할 디렉토리

    merge_faiss_databases(db1_dir, db2_dir, merged_dir)


AssertionError: docstore1가 dict가 아닙니다: <class 'langchain_community.docstore.in_memory.InMemoryDocstore'>

In [6]:
import pickle
from pathlib import Path
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

def load_local_custom(folder_path, embeddings, index_name="index"):
    """
    저장된 FAISS DB 디렉토리에서 index.faiss와 index.pkl 파일을 로드합니다.
    pickle 파일이 튜플이면 처음 2개의 값을, 딕셔너리이면 
    "docstore"와 "index_to_docstore_id" 키의 값을 사용합니다.
    """
    folder = Path(folder_path)
    index = faiss.read_index(str(folder / "index.faiss"))
    pkl_path = folder / f"{index_name}.pkl"
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)
    # data가 튜플인 경우
    if isinstance(data, tuple):
        if len(data) >= 2:
            docstore, index_to_docstore_id = data[0], data[1]
        else:
            raise ValueError("피클 튜플의 길이가 2보다 작습니다.")
    # data가 딕셔너리인 경우 (예: {"docstore": ..., "index_to_docstore_id": ...})
    elif isinstance(data, dict):
        if "docstore" in data and "index_to_docstore_id" in data:
            docstore = data["docstore"]
            index_to_docstore_id = data["index_to_docstore_id"]
        else:
            raise ValueError("피클 딕셔너리에 'docstore' 또는 'index_to_docstore_id' 키가 없습니다.")
    else:
        raise ValueError("피클 파일의 형식이 예상과 다릅니다.")
    
    return FAISS(embeddings, index, docstore, index_to_docstore_id)

# OpenAIEmbeddings 인스턴스 생성 (API 키 등 설정 필요)
embeddings = OpenAIEmbeddings()

# 저장된 FAISS DB 로드 (각 디렉토리에 저장된 index.faiss, index.pkl 파일 사용)
db1 = load_local_custom("faiss_artworks_0304", embeddings, index_name="index")
db2 = load_local_custom("faiss_artworks_0318", embeddings, index_name="index")

# db2의 벡터들을 db1에 병합 (merge_from 메서드 사용)
db1.merge_from(db2)

# 병합된 FAISS DB를 새 디렉토리에 저장
db1.save_local("faiss_merged_db")

print("병합 완료!")


ValueError: 피클 딕셔너리에 'docstore' 또는 'index_to_docstore_id' 키가 없습니다.

In [35]:
print(f"문서 ID: {list(docstore.keys())[10]}")
print(f"첫 번째 문서 내용: {docstore[list(docstore.keys())[10]].page_content[:100]}...")

AttributeError: 'InMemoryDocstore' object has no attribute 'keys'

In [44]:
query = "오지호 풍경 하부 도상의 제작연도는 어떻게 추정해?"

In [45]:
response = chain.invoke({"question": query})
print(response) 

ValueError: Could not find document for id 498, got ID 498 not found.

In [ ]:
# 매핑된 문서 ID 확인
for vector_id in range(faiss_index.ntotal):
    if vector_id in index_to_docstore_id:
        doc_id = index_to_docstore_id[vector_id]
        print(f"벡터 ID: {vector_id}, 문서 ID: {doc_id}, 문서 내용: {docstore[doc_id].page_content[:100]}...")
    else:
        print(f"벡터 ID: {vector_id}에 대한 문서 ID가 매핑되지 않았습니다.")
